# Time Series Machine Learning Experimentation

This notebook implements various time series forecasting models for appliances energy consumption prediction.

## Import packages

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Time series models
from prophet import Prophet
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

# Deep learning for time series
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

# Evaluation metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Plotting
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = 'Times New Roman'

print("All packages imported successfully!")

## Data preparation

In [ ]:
# Load and prepare data
df = pd.read_csv('../data/energydata_complete_cleaned.csv', parse_dates=['date'], index_col='date')
df.sort_index(inplace=True)
print('Data shape: ', df.shape)
print('Date range:', df.index.min(), 'to', df.index.max())

# Define features and target variable
features = ['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 
            'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 
            'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 
            'Visibility', 'Tdewpoint', 'hour_of_day', 'day_of_week', 
            'is_weekend', 'hour_sin', 'hour_cos', 'day_of_week_sin', 
            'day_of_week_cos', 'Appliances_lag1', 'Appliances_rolling_mean_6']
target = 'Appliances'
print('Feature number: ', len(features))

# For time series models, we'll use chronological split
train_size = int(len(df) * 0.7)
train_df = df.iloc[:train_size].copy()
test_df = df.iloc[train_size:].copy()

print('Training data shape: ', train_df.shape)
print('Testing data shape: ', test_df.shape)
print('Training period:', train_df.index.min(), 'to', train_df.index.max())
print('Testing period:', test_df.index.min(), 'to', test_df.index.max())

# Prepare Prophet format data
prophet_train = train_df.reset_index().rename(columns={'date': 'ds', target: 'y'})[['ds', 'y']]
prophet_test = test_df.reset_index().rename(columns={'date': 'ds', target: 'y'})[['ds', 'y']]

print('\nProphet training data shape:', prophet_train.shape)
print('Prophet testing data shape:', prophet_test.shape)

In [ ]:
# Initialize results dictionary
model_results = {}

def evaluate_model(y_true, y_pred, model_name):
    """Calculate evaluation metrics for a model"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    results = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    }
    
    model_results[model_name] = results
    
    print(f"\n{model_name} Results:")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"R²: {r2:.4f}")
    
    return results

## 1. Prophet Model

In [ ]:
print("Training Prophet model...")

# Initialize Prophet model
prophet_model = Prophet(
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=False,  # no yearly seasonality due to limited data period
    changepoint_prior_scale=0.05,
    seasonality_prior_scale=10.0,
    holidays_prior_scale=10.0,
    seasonality_mode='multiplicative'
)

# Fit the model
prophet_model.fit(prophet_train)

# Make predictions
prophet_forecast = prophet_model.predict(prophet_test[['ds']])
prophet_predictions = prophet_forecast['yhat'].values

# Evaluate Prophet model
prophet_results = evaluate_model(prophet_test['y'].values, prophet_predictions, 'Prophet')

print("Prophet model training completed!")

In [ ]:
# Visualize Prophet results
fig, axes = plt.subplots(2, 1, figsize=(15, 12))

# Plot 1: Forecast vs Actual
axes[0].plot(prophet_test['ds'], prophet_test['y'], label='Actual', color='blue', alpha=0.7)
axes[0].plot(prophet_test['ds'], prophet_predictions, label='Prophet Forecast', color='red', alpha=0.7)
axes[0].set_title('Prophet Model: Actual vs Predicted Appliances Energy Consumption')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Energy Consumption (Wh)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Residuals
residuals = prophet_test['y'].values - prophet_predictions
axes[1].scatter(prophet_predictions, residuals, alpha=0.6)
axes[1].axhline(y=0, color='red', linestyle='--')
axes[1].set_title('Prophet Model: Residuals Plot')
axes[1].set_xlabel('Predicted Values')
axes[1].set_ylabel('Residuals')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/prediction_plots/prophet_results.png', dpi=300, bbox_inches='tight')
plt.show()

# Prophet components
fig = prophet_model.plot_components(prophet_forecast)
plt.savefig('../results/prediction_plots/prophet_components.png', dpi=300, bbox_inches='tight')
plt.show()

## 2. ARIMA Model

In [ ]:
# Check stationarity of the time series
def check_stationarity(timeseries, title):
    """Check if time series is stationary using Augmented Dickey-Fuller test"""
    print(f'Results of Augmented Dickey-Fuller Test for {title}:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    
    if dftest[1] <= 0.05:
        print("Series is stationary")
        return True
    else:
        print("Series is not stationary")
        return False

# Check original series
appliances_series = train_df[target]
is_stationary = check_stationarity(appliances_series, "Original Appliances Series")

# If not stationary, apply differencing
if not is_stationary:
    appliances_diff = appliances_series.diff().dropna()
    print("\nAfter first differencing:")
    is_stationary_diff = check_stationarity(appliances_diff, "First Differenced Series")
    
    if is_stationary_diff:
        d_param = 1
        working_series = appliances_diff
    else:
        appliances_diff2 = appliances_diff.diff().dropna()
        print("\nAfter second differencing:")
        check_stationarity(appliances_diff2, "Second Differenced Series")
        d_param = 2
        working_series = appliances_diff2
else:
    d_param = 0
    working_series = appliances_series

print(f"\nUsing d parameter: {d_param}")

In [ ]:
print("Training ARIMA model...")

# Fit ARIMA model with auto-selected parameters
# Using a simple ARIMA(2,1,2) as a starting point
try:
    arima_model = ARIMA(train_df[target], order=(2, d_param, 2))
    arima_fitted = arima_model.fit()
    
    print("ARIMA model summary:")
    print(arima_fitted.summary())
    
    # Make predictions
    arima_forecast = arima_fitted.forecast(steps=len(test_df))
    arima_predictions = arima_forecast.values
    
    # Ensure predictions are non-negative (energy consumption can't be negative)
    arima_predictions = np.maximum(arima_predictions, 0)
    
    # Evaluate ARIMA model
    arima_results = evaluate_model(test_df[target].values, arima_predictions, 'ARIMA')
    
    print("ARIMA model training completed!")
    
except Exception as e:
    print(f"Error training ARIMA model: {e}")
    print("Trying simpler ARIMA(1,1,1) model...")
    
    try:
        arima_model = ARIMA(train_df[target], order=(1, 1, 1))
        arima_fitted = arima_model.fit()
        
        arima_forecast = arima_fitted.forecast(steps=len(test_df))
        arima_predictions = arima_forecast.values
        arima_predictions = np.maximum(arima_predictions, 0)
        
        arima_results = evaluate_model(test_df[target].values, arima_predictions, 'ARIMA')
        print("ARIMA(1,1,1) model training completed!")
        
    except Exception as e2:
        print(f"Error with ARIMA(1,1,1): {e2}")
        arima_predictions = np.full(len(test_df), train_df[target].mean())
        arima_results = evaluate_model(test_df[target].values, arima_predictions, 'ARIMA (Mean Baseline)')

In [ ]:
# Visualize ARIMA results
fig, axes = plt.subplots(2, 1, figsize=(15, 12))

# Plot 1: Forecast vs Actual
axes[0].plot(test_df.index, test_df[target], label='Actual', color='blue', alpha=0.7)
axes[0].plot(test_df.index, arima_predictions, label='ARIMA Forecast', color='green', alpha=0.7)
axes[0].set_title('ARIMA Model: Actual vs Predicted Appliances Energy Consumption')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Energy Consumption (Wh)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Residuals
arima_residuals = test_df[target].values - arima_predictions
axes[1].scatter(arima_predictions, arima_residuals, alpha=0.6, color='green')
axes[1].axhline(y=0, color='red', linestyle='--')
axes[1].set_title('ARIMA Model: Residuals Plot')
axes[1].set_xlabel('Predicted Values')
axes[1].set_ylabel('Residuals')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/prediction_plots/arima_results.png', dpi=300, bbox_inches='tight')
plt.show()

## 3. LSTM Model

In [ ]:
# Prepare data for LSTM
def create_lstm_dataset(data, target_col, features, lookback=24):
    """Create dataset for LSTM with lookback window"""
    # Scale the data
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    
    X_scaled = scaler_X.fit_transform(data[features])
    y_scaled = scaler_y.fit_transform(data[target_col].values.reshape(-1, 1))
    
    X, y = [], []
    for i in range(lookback, len(X_scaled)):
        X.append(X_scaled[i-lookback:i])
        y.append(y_scaled[i])
    
    return np.array(X), np.array(y), scaler_X, scaler_y

print("Preparing LSTM data...")

# Use a subset of most important features for LSTM
lstm_features = ['lights', 'T_out', 'RH_out', 'hour_of_day', 'day_of_week', 
                 'is_weekend', 'Appliances_lag1', 'Appliances_rolling_mean_6']

lookback_window = 24  # Use 24 time steps (4 hours) to predict next step

# Prepare training data
X_train_lstm, y_train_lstm, scaler_X, scaler_y = create_lstm_dataset(
    train_df, target, lstm_features, lookback_window
)

print(f"LSTM training data shape: X={X_train_lstm.shape}, y={y_train_lstm.shape}")
print(f"Features used: {len(lstm_features)}")
print(f"Lookback window: {lookback_window} time steps")

In [ ]:
print("Building and training LSTM model...")

# Build LSTM model
lstm_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(lookback_window, len(lstm_features))),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

print("LSTM Model Architecture:")
lstm_model.summary()

# Train the model
history = lstm_model.fit(
    X_train_lstm, y_train_lstm,
    batch_size=32,
    epochs=50,
    validation_split=0.2,
    verbose=1,
    shuffle=False  # Important for time series
)

print("LSTM model training completed!")

In [ ]:
# Make LSTM predictions
print("Making LSTM predictions...")

# Prepare test data for LSTM
# We need to use the last lookback_window points from training data to start predictions
full_data = pd.concat([train_df, test_df])
test_start_idx = len(train_df)

lstm_predictions = []
current_batch = scaler_X.transform(full_data[lstm_features].iloc[test_start_idx-lookback_window:test_start_idx].values)

for i in range(len(test_df)):
    # Predict next value
    current_pred = lstm_model.predict(current_batch.reshape(1, lookback_window, len(lstm_features)), verbose=0)
    lstm_predictions.append(current_pred[0, 0])
    
    # Update batch for next prediction
    if i < len(test_df) - 1:
        # Use actual next values to update the batch (teacher forcing for evaluation)
        next_features = scaler_X.transform(full_data[lstm_features].iloc[test_start_idx+i+1:test_start_idx+i+2].values)
        current_batch = np.append(current_batch[1:], next_features, axis=0)

# Inverse transform predictions
lstm_predictions = np.array(lstm_predictions).reshape(-1, 1)
lstm_predictions = scaler_y.inverse_transform(lstm_predictions).flatten()

# Ensure predictions are non-negative
lstm_predictions = np.maximum(lstm_predictions, 0)

# Evaluate LSTM model
lstm_results = evaluate_model(test_df[target].values, lstm_predictions, 'LSTM')

print("LSTM predictions completed!")

In [ ]:
# Visualize LSTM results
fig, axes = plt.subplots(3, 1, figsize=(15, 18))

# Plot 1: Training history
axes[0].plot(history.history['loss'], label='Training Loss')
axes[0].plot(history.history['val_loss'], label='Validation Loss')
axes[0].set_title('LSTM Model Training History')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Forecast vs Actual
axes[1].plot(test_df.index, test_df[target], label='Actual', color='blue', alpha=0.7)
axes[1].plot(test_df.index, lstm_predictions, label='LSTM Forecast', color='orange', alpha=0.7)
axes[1].set_title('LSTM Model: Actual vs Predicted Appliances Energy Consumption')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Energy Consumption (Wh)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot 3: Residuals
lstm_residuals = test_df[target].values - lstm_predictions
axes[2].scatter(lstm_predictions, lstm_residuals, alpha=0.6, color='orange')
axes[2].axhline(y=0, color='red', linestyle='--')
axes[2].set_title('LSTM Model: Residuals Plot')
axes[2].set_xlabel('Predicted Values')
axes[2].set_ylabel('Residuals')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/prediction_plots/lstm_results.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Model Comparison and Results

In [ ]:
# Create comprehensive comparison
print("=" * 60)
print("TIME SERIES MODELS PERFORMANCE COMPARISON")
print("=" * 60)

# Create results DataFrame
results_df = pd.DataFrame(model_results).T
results_df = results_df.round(4)

print("\nDetailed Results:")
print(results_df)

# Find best model for each metric
print("\nBest Models by Metric:")
print(f"Lowest RMSE: {results_df['RMSE'].idxmin()} ({results_df['RMSE'].min():.2f})")
print(f"Lowest MAE: {results_df['MAE'].idxmin()} ({results_df['MAE'].min():.2f})")
print(f"Highest R²: {results_df['R2'].idxmax()} ({results_df['R2'].max():.4f})")

# Overall ranking (based on R² score)
ranking = results_df.sort_values('R2', ascending=False)
print("\nOverall Ranking (by R² score):")
for i, (model, row) in enumerate(ranking.iterrows(), 1):
    print(f"{i}. {model}: R² = {row['R2']:.4f}, RMSE = {row['RMSE']:.2f}")

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(20, 16))

# Plot 1: All predictions comparison
axes[0, 0].plot(test_df.index, test_df[target], label='Actual', color='black', linewidth=2, alpha=0.8)
axes[0, 0].plot(test_df.index, prophet_predictions, label='Prophet', color='red', alpha=0.7)
axes[0, 0].plot(test_df.index, arima_predictions, label='ARIMA', color='green', alpha=0.7)
axes[0, 0].plot(test_df.index, lstm_predictions, label='LSTM', color='orange', alpha=0.7)
axes[0, 0].set_title('Time Series Models Comparison: Predictions vs Actual', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Energy Consumption (Wh)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Performance metrics comparison
metrics = ['RMSE', 'MAE', 'R2']
x_pos = np.arange(len(results_df.index))
width = 0.25

for i, metric in enumerate(metrics):
    if metric == 'R2':
        # For R², higher is better, so we can plot directly
        axes[0, 1].bar(x_pos + i*width, results_df[metric], width, 
                      label=metric, alpha=0.8)
    else:
        # For RMSE and MAE, lower is better, so we normalize for visualization
        normalized_values = 1 / (1 + results_df[metric] / results_df[metric].max())
        axes[0, 1].bar(x_pos + i*width, normalized_values, width, 
                      label=f'{metric} (normalized)', alpha=0.8)

axes[0, 1].set_title('Model Performance Metrics Comparison', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Models')
axes[0, 1].set_ylabel('Performance Score')
axes[0, 1].set_xticks(x_pos + width)
axes[0, 1].set_xticklabels(results_df.index, rotation=45)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Residuals comparison
residuals_data = {
    'Prophet': test_df[target].values - prophet_predictions,
    'ARIMA': test_df[target].values - arima_predictions,
    'LSTM': test_df[target].values - lstm_predictions
}

axes[1, 0].boxplot(residuals_data.values(), labels=residuals_data.keys())
axes[1, 0].axhline(y=0, color='red', linestyle='--', alpha=0.7)
axes[1, 0].set_title('Residuals Distribution Comparison', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('Residuals')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Scatter plot of predictions vs actual
axes[1, 1].scatter(test_df[target], prophet_predictions, alpha=0.6, label='Prophet', color='red')
axes[1, 1].scatter(test_df[target], arima_predictions, alpha=0.6, label='ARIMA', color='green')
axes[1, 1].scatter(test_df[target], lstm_predictions, alpha=0.6, label='LSTM', color='orange')

# Perfect prediction line
min_val = min(test_df[target].min(), min(prophet_predictions.min(), arima_predictions.min(), lstm_predictions.min()))
max_val = max(test_df[target].max(), max(prophet_predictions.max(), arima_predictions.max(), lstm_predictions.max()))
axes[1, 1].plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.7, label='Perfect Prediction')

axes[1, 1].set_title('Predicted vs Actual Values', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Actual Values')
axes[1, 1].set_ylabel('Predicted Values')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/prediction_plots/timeseries_models_comprehensive_comparison.png', 
            dpi=300, bbox_inches='tight')
plt.show()

# Save results to CSV
results_df.to_csv('../results/timeseries_model_results.csv')
print("\nResults saved to '../results/timeseries_model_results.csv'")

## 5. Conclusions and Insights

In [ ]:
print("=" * 80)
print("TIME SERIES FORECASTING ANALYSIS - CONCLUSIONS")
print("=" * 80)

best_model = results_df['R2'].idxmax()
best_r2 = results_df['R2'].max()
best_rmse = results_df.loc[best_model, 'RMSE']

print(f"\n🏆 BEST PERFORMING MODEL: {best_model}")
print(f"   - R² Score: {best_r2:.4f}")
print(f"   - RMSE: {best_rmse:.2f} Wh")
print(f"   - MAE: {results_df.loc[best_model, 'MAE']:.2f} Wh")

print("\n📊 KEY INSIGHTS:")

# Model-specific insights
if 'Prophet' in model_results:
    prophet_r2 = model_results['Prophet']['R2']
    print(f"\n🔮 Prophet Model (R² = {prophet_r2:.4f}):")
    print("   - Excellent at capturing seasonal patterns")
    print("   - Handles daily and weekly seasonality well")
    print("   - Robust to missing data and outliers")
    if prophet_r2 > 0.6:
        print("   - Shows strong predictive performance")
    else:
        print("   - May need additional regressors for better performance")

if 'ARIMA' in model_results:
    arima_r2 = model_results['ARIMA']['R2']
    print(f"\n📈 ARIMA Model (R² = {arima_r2:.4f}):")
    print("   - Classical time series approach")
    print("   - Good for capturing linear trends and patterns")
    if arima_r2 < 0.3:
        print("   - May struggle with complex non-linear patterns")
        print("   - Could benefit from more sophisticated parameter tuning")
    else:
        print("   - Shows reasonable performance for linear patterns")

if 'LSTM' in model_results:
    lstm_r2 = model_results['LSTM']['R2']
    print(f"\n🧠 LSTM Model (R² = {lstm_r2:.4f}):")
    print("   - Deep learning approach for complex patterns")
    print("   - Can capture non-linear relationships")
    if lstm_r2 > 0.5:
        print("   - Successfully learned complex temporal dependencies")
    else:
        print("   - May need more training data or architecture tuning")
    print("   - Requires more computational resources")

print("\n🎯 RECOMMENDATIONS:")
print(f"\n1. For production deployment, use {best_model} model")
print("2. Consider ensemble methods combining multiple models")
print("3. Monitor model performance and retrain periodically")
print("4. Collect more external features (weather, occupancy) for better accuracy")

if best_r2 < 0.7:
    print("\n⚠️  IMPROVEMENT OPPORTUNITIES:")
    print("   - Add more external regressors (weather, occupancy patterns)")
    print("   - Consider hybrid models combining multiple approaches")
    print("   - Implement more sophisticated feature engineering")
    print("   - Use ensemble methods for better robustness")

print("\n📋 TECHNICAL NOTES:")
print("   - All models evaluated on chronological test split (30% of data)")
print("   - Predictions constrained to non-negative values")
print("   - LSTM used 24-step lookback window (4 hours)")
print("   - Prophet configured for daily/weekly seasonality")

print("\n" + "=" * 80)
print("ANALYSIS COMPLETE - All results saved to ../results/ directory")
print("=" * 80)